Lowest Error for Each Metabolite as Discrete variables

Error defined as percentage of samples mislabeled


0: Logistic Regression, Linear SVC

1: Logistic Regression, Perceptron, Linear SVC

2: Linear SVC

3: Logistic Regression, Perceptron, Linear SVC

4: Logistic Regression

5: Linear SVC

6: Logistic Regression, Linear SVC

7: Logistic Regression

8: Linear SVC

9: Logistic Regression

10: Logistic Regression

11: Perceptron, Linear SVC

12: Logistic Regression, Perceptron, Linear SVC

13: Linear SVC


An attempt at solving for continuous variables was made, but a switch to discrete variables was made due to time.


Linear SVC had the most counts for lowest error.

In [1]:
# Imports
import random
import numpy
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [2]:
# Randomly separating inputs into training and test sets
def prepData(inFile, outFile):
    # Opening specified files
    inRaw = open(inFile, 'r')
    outRaw = open(outFile, 'r')
    inSampleRaw = inRaw.readlines()
    outSampleRaw = outRaw.readlines()
    
    # Initializing arrays
    inRawArray = []
    outRawArray = []
    
    for sample in range(len(inSampleRaw)):
        inSample = inSampleRaw[sample].split(",")
        outSample = outSampleRaw[sample].split(",")

        if(inSample[0] != ""):
            inRawArray.append(inSample[1:])
            outRawArray.append(outSample[1:])
            
            # Cleaning up newline
            if("\n" in inRawArray[-1][-1]):
                inRawArray[-1][-1] = (inRawArray[-1][-1])[:-2]
            if("\n" in outRawArray[-1][-1]):
                outRawArray[-1][-1] = (outRawArray[-1][-1])[:-2]
            
            inRawArray[-1] = [float(value) for value in inRawArray[-1]]
            outRawArray[-1] = [float(value) for value in outRawArray[-1]]            
            


    
    # Total Sample count
    sampleSize = len(inRawArray)
    
    # Creating discrete output sets (Binary for now)
    outDiscreteRaw = []
    for row in outRawArray:
        outDiscreteRaw.append([-1 if col < 0 else 1 for col in row])
            
    # Randomly generating training and test in 70/30 split
    random.seed(111) # Setting seed
    testSamples = random.sample(range(sampleSize), round(sampleSize * 0.3))
    testSamples.sort()
    trainSamples = [x for x in range(sampleSize) if x not in testSamples]
    
    inTrain = numpy.zeros((sampleSize - len(testSamples), len(inRawArray[0])), dtype=numpy.float32)
    inTest = numpy.zeros((len(testSamples), len(inRawArray[0])), dtype=numpy.float32)
    outTrain = numpy.zeros((sampleSize - len(testSamples), len(outRawArray[0])), dtype=numpy.float32)
    outTest = numpy.zeros((len(testSamples), len(outRawArray[0])), dtype=numpy.float32)
    outDiscreteTrain = numpy.zeros((sampleSize - len(testSamples), len(outRawArray[0])), dtype=numpy.float32)
    outDiscreteTest = numpy.zeros((len(testSamples), len(outRawArray[0])), dtype=numpy.float32)
    
    
    inTrain[:,] = [inRawArray[i] for i in trainSamples]
    inTest[:,] = [inRawArray[i] for i in testSamples]
    outTrain[:,] = [outRawArray[i] for i in trainSamples]
    outTest[:,] = [outRawArray[i] for i in testSamples]
    outDiscreteTrain[:,] = [outDiscreteRaw[i] for i in trainSamples]
    outDiscreteTest[:,] = [outDiscreteRaw[i] for i in testSamples]
    
    inRaw.close()
    outRaw.close()
    
    return inTrain, inTest, outTrain, outTest, outDiscreteTrain, outDiscreteTest
    
inTrain, inTest, outTrain, outTest, disTrain, disTest = prepData("protein_input.csv", "met_output.csv")


In [3]:
# Logistic regression with Discrete output with GridSearch results
# Validated with first column of output
model = LogisticRegression(penalty="l1", fit_intercept=False, solver="liblinear")
modelFit = model.fit(inTrain, disTrain[:, 0])
testPredict = modelFit.predict(inTest)

for col in range(len(disTest[0])):
    error = 0
    for sample in range(len(disTest[:, col])):
        error += numpy.abs(disTest[sample, col] - testPredict[sample]) / 2
    print(col, "Error:", error / len(outTest))


0 Error: 0.20192307692307693
1 Error: 0.5288461538461539
2 Error: 0.3076923076923077
3 Error: 0.41346153846153844
4 Error: 0.38461538461538464
5 Error: 0.6153846153846154
6 Error: 0.3557692307692308
7 Error: 0.41346153846153844
8 Error: 0.4423076923076923
9 Error: 0.25961538461538464
10 Error: 0.6153846153846154
11 Error: 0.5673076923076923
12 Error: 0.49038461538461536
13 Error: 0.49038461538461536


In [4]:
# Logistic regression with Discrete output GridSearch
parameterDictionary = {

    "penalty":["none", "l2", "l1", "elasticnet"],

    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],

    "fit_intercept":[True, False]

    }
model = LogisticRegression(max_iter=1000)
GSCV = GridSearchCV(model, parameterDictionary)
modelFit = GSCV.fit(inTrain, disTrain[:, 0])

testPredict = modelFit.predict(inTest)

for key in GSCV.best_params_:      
    print("For", key, ":", GSCV.best_params_[key])

error = 0
for sample in range(len(disTest[:, 0])):
    error += numpy.abs(disTest[sample, 0] - testPredict[sample])
print("Error:", error / len(outTest[:, 0]))

c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py", line 454, in _check_solver
    "penalty='none' is not supported for the liblinear solver"
ValueError: penalty='none' is not supported for the liblinear solver

  FitFailedWarning)
c:\users\ktran\appdata\lo

c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py", line 454, in _check_solver
    "penalty='none' is not supported for the liblinear solver"
ValueError: penalty='none' is not supported for the liblinear solver

  FitFailedWarning)
c:\users\ktran\appdata\lo

For fit_intercept : False
For penalty : l1
For solver : liblinear
Error: 0.40384615384615385


c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages

In [5]:
# Linear regression with Discrete output
# Validated with first column of output
model = LinearRegression()
modelFit = model.fit(inTrain, disTrain[:, 0])
testPredict = modelFit.predict(inTest)

for col in range(len(disTest[0])):
    error = 0
    for sample in range(len(disTest[:, col])):
        error += numpy.abs(disTest[sample, col] - testPredict[sample]) / 2
    print(col, "Error:", error / len(outTest))

0 Error: 0.4391936691334614
1 Error: 0.6360060979540532
2 Error: 0.48362468813474363
3 Error: 0.5768723648328048
4 Error: 0.5533908255971395
5 Error: 0.6940003527471652
6 Error: 0.5875357701801337
7 Error: 0.6292677983068503
8 Error: 0.5793729751155927
9 Error: 0.49875400186731267
10 Error: 0.7503455344301003
11 Error: 0.645317252438802
12 Error: 0.6316494738253263
13 Error: 0.6428780885270009


In [6]:
# Perceptron with Discrete output with GridSearch Results
# Validated with first column of output
model = Perceptron(alpha=0.001, fit_intercept=False, penalty="l1")
modelFit = model.fit(inTrain, disTrain[:, 0])
testPredict = modelFit.predict(inTest)

for col in range(len(disTest[0])):
    error = 0
    for sample in range(len(disTest[:, col])):
        error += numpy.abs(disTest[sample, col] - testPredict[sample]) / 2
    print(col, "Error:", error / len(outTest))

0 Error: 0.2403846153846154
1 Error: 0.5288461538461539
2 Error: 0.3076923076923077
3 Error: 0.41346153846153844
4 Error: 0.40384615384615385
5 Error: 0.6153846153846154
6 Error: 0.3942307692307692
7 Error: 0.4326923076923077
8 Error: 0.4423076923076923
9 Error: 0.3557692307692308
10 Error: 0.6346153846153846
11 Error: 0.5480769230769231
12 Error: 0.49038461538461536
13 Error: 0.5288461538461539


In [7]:
# Perceptron with Discrete output GridSearch
parameterDictionary = {

    "penalty":["l2", "l1", "elasticnet"],
    
    "alpha": [0.001, 0.0001, 0.00001], 

    "fit_intercept":[True, False]

    }
model = Perceptron()
GSCV = GridSearchCV(model, parameterDictionary)
modelFit = GSCV.fit(inTrain, disTrain[:, 0])
testPredict = modelFit.predict(inTest)

for key in GSCV.best_params_:      
    print("For", key, ":", GSCV.best_params_[key])

For alpha : 0.001
For fit_intercept : False
For penalty : l1


In [8]:
# LinearSVC with Discrete output with GridSearch Results
# Validated with first column of output
model = LinearSVC(penalty="l1", loss="squared_hinge", dual=False, max_iter=10000)
modelFit = model.fit(inTrain, disTrain[:, 0])
testPredict = modelFit.predict(inTest)

for col in range(len(disTest[0])):
    error = 0
    for sample in range(len(disTest[:, col])):
        error += numpy.abs(disTest[sample, col] - testPredict[sample]) / 2
    print(col, "Error:", error / len(outTest))

0 Error: 0.20192307692307693
1 Error: 0.5288461538461539
2 Error: 0.28846153846153844
3 Error: 0.41346153846153844
4 Error: 0.40384615384615385
5 Error: 0.5961538461538461
6 Error: 0.3557692307692308
7 Error: 0.4326923076923077
8 Error: 0.4230769230769231
9 Error: 0.3173076923076923
10 Error: 0.6538461538461539
11 Error: 0.5480769230769231
12 Error: 0.49038461538461536
13 Error: 0.47115384615384615


In [9]:
# LinearSVC with Discrete output GridSearch
parameterDictionary = {

    "penalty":["l2", "l1"],
    
    "loss": ["hinge", "squared_hinge"],
    
    "dual": [True, False]

    }
model = LinearSVC(max_iter=10000)
GSCV = GridSearchCV(model, parameterDictionary)
modelFit = GSCV.fit(inTrain, disTrain[:, 0])
testPredict = modelFit.predict(inTest)

for key in GSCV.best_params_:      
    print("For", key, ":", GSCV.best_params_[key])

c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_classes.py", line 237, in fit
    self.loss, sample_weight=sample_weight)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py", line 965, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "c:\users\ktran\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py", line 823, in _get_liblinear_solver_type


For dual : False
For loss : squared_hinge
For penalty : l1
